# Segment clouds and cloud shadows in Landsat images (L2A)
This notebook shows an example on how to use [ukis-csmask]() to segment clouds and cloud shadows in Level-2A images from Landsat-9, Landsat-8, Landsat-7 and Landsat-5 satellites. Images are acquired from [Planetary Computer]() and are prepared according to [ukis-csmask]() requirements.

> NOTE: to run this notebook, we first need to install some additional dependencies for work with Planetary Computer
```shell
pip install planetary_computer pystac-client odc-stac tqdm
pip install -e git+https://github.com/dlr-eoc/ukis-csmask/tree/l2a#egg=ukis-csmask[gpu]
```

In [1]:
import numpy as np
import planetary_computer as pc

from odc.stac import load
from pystac_client import Client
from tqdm import tqdm
from ukis_csmask.mask import CSmask

In [13]:
stac_api_endpoint = "https://planetarycomputer.microsoft.com/api/stac/v1"
collections = ["landsat-c2-l2"]
ids = [
    "LC09_L2SP_194027_20240604_02_T1",
    "LC08_L2SP_191027_20240607_02_T1",
    "LE07_L2SP_192027_20050805_02_T1",
    "LT05_L2SP_192026_20100726_02_T1",
]
bbox = [11.016, 47.666, 12.238, 48.472]
product_level = "l2a"
band_order = ["blue", "green", "red", "nir", "swir16", "swir22"]
providers = ["CUDAExecutionProvider"]

In [15]:
# search catalog by scene id
catalog = Client.open(stac_api_endpoint)
search = catalog.search(collections=collections, ids=ids)
items = [item for item in search.items()]
items_cnt = len(items)
print(f"Search returned {items_cnt} item(s)")

Search returned 4 item(s)


In [18]:
import rioxarray

for item in tqdm(items, total=items_cnt, desc="Predict images"):
    # near infrared band has different alias in landsat collections
    bands = [b.replace("nir", "nir08") for b in band_order]

    # load and preprocess image
    arr = (
        load(
            items=[item],
            bands=bands,
            resolution=30,
            bbox=bbox,
            dtype="float32",
            patch_url=pc.sign,
        )
        .to_dataarray()
        .squeeze()
        .drop_vars("time")
    )
    arr = arr.rename({"variable": "band"})
    arr.rio.to_raster(raster_path="l8.tif", driver="COG")

    # use band-specific rescale factors to convert DN to reflectance
    for idx, band_name in enumerate(bands):
        band_info = item.assets[band_name].extra_fields["raster:bands"][0]
        arr[idx, :, :] = arr.sel(band=str(band_name)).astype(np.float32) * band_info["scale"]
        arr[idx, :, :] += band_info["offset"]
        arr[idx, :, :] = arr[idx, :, :].clip(min=0.0, max=1.0)
    print(arr.shape, arr.dtype)

    # compute cloud and cloud shadow mask
    csmask = CSmask(
        img=np.moveaxis(arr.to_numpy(), 0, -1),
        band_order=band_order,
        product_level=product_level,
        nodata_value=0,
        invalid_buffer=4,
        intra_op_num_threads=0,
        inter_op_num_threads=0,
        providers=providers,
        batch_size=1,
    )

    # access cloud and cloud shadow mask
    csmask_csm = csmask.csm

    # access valid mask
    csmask_valid = csmask.valid

Predict images:   0%|          | 0/4 [00:00<?, ?it/s]

Predict images:   0%|          | 0/4 [02:25<?, ?it/s]

(6, 6038, 7076) float32


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data1/Workspace/python/ukis-csmask/src/ukis-csmask/ukis_csmask/model_6b_l2a.json'